In [17]:
import pandas as pd
import datetime
import warnings
import numpy as np
import pickle as pkl
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from catboost import Pool, cv
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTENC
from xgboost import XGBRegressor
from pandas_profiling import ProfileReport
import xgboost as xgb
from chinese_calendar import is_workday
import lightgbm as lgb
import optuna
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号

In [18]:
PATH = os.path.realpath(r"E:\document\code\python\02-competition\E-commerce-Sales-Forecast")
dataset_path = os.path.join(PATH, "dataset")

In [19]:
def generate_dataset(data_df: pd.DataFrame, cols=None):
    if cols is None:
        cols = ['总销量', '直播个数', '直播销量', '视频销量', '直播达人', '直播销量占比', '视频销量占比']
    remove_cols = list()
    data_df["时间"] = data_df["时间"].apply(lambda x: str(x).split(" ")[0])
    data_df["时间"] = pd.to_datetime(data_df["时间"])
    data_df["星期数"] = data_df["时间"].dt.isocalendar().week
    data_df["星期数"] = data_df["星期数"].astype(int)
    data_df["直播销量占比"] = data_df.apply(lambda x: 0.0 if x["总销量"] == 0 else x["直播销量"] / x["总销量"], axis=1)
    data_df["视频销量占比"] = data_df.apply(lambda x: 0.0 if x["总销量"] == 0 else x["视频销量"] / x["总销量"], axis=1)
    backward_list = list()
    for name in cols:
        forward_list = list()
        for _idx in range(1, 15):
            data_df[name + "_forward_" + str(_idx)] = data_df[name].shift(periods=_idx, fill_value=0)
            remove_cols.append(name + "_forward_" + str(_idx))
            forward_list.append(name + "_forward_" + str(_idx))
        forward_list.append(name)
        tmp_df_1 = data_df[forward_list[:7]]
        tmp_df_2 = data_df[forward_list[7:]]
        tmp_df = data_df[forward_list]
        data_df[name + "_前第一周天平均值"] = tmp_df_1.mean(axis=1)
        data_df[name + "_前第二周天平均值"] = tmp_df_2.mean(axis=1)
        data_df[name + "_前第一周天方差值"] = tmp_df_1.mean(axis=1)
        data_df[name + "_前第二周天方差值"] = tmp_df_2.mean(axis=1)
        data_df[name + "_平均值增量"] = data_df[name + "_前第一周天平均值"] - data_df[name + "_前第二周天平均值"]
        data_df[name + "_方差值增量"] = data_df[name + "_前第一周天方差值"] - data_df[name + "_前第二周天方差值"]
        data_df[name + "_前两周最大值"] = tmp_df.max(axis=1)
        data_df[name + "_前两周最小值"] = tmp_df.min(axis=1)
        if name == "总销量":
            for j in range(1, 8):
                data_df[name + "_backward_" + str(j)] = data_df[name].shift(periods=j * (-1), fill_value=0)
                backward_list.append(name + "_backward_" + str(j))
                remove_cols.append(name + "_backward_" + str(j))
            tmp_df = data_df[backward_list]
            data_df["后七天平均销量"] = tmp_df.mean(axis=1)
        # remove_cols.append(name)
    for _idx, date_time in enumerate(
            ["2021-12-26", "2021-12-25", "2021-12-24", "2021-12-23", "2021-12-22", "2021-12-21"]):
        data_df.loc[data_df[data_df["时间"] == date_time].index, "后七天平均销量"] = data_df.loc[
            data_df[data_df["时间"] == date_time].index, backward_list[:_idx + 1]].mean(axis=1)
    data_df.drop(columns=remove_cols, inplace=True)

In [20]:
df_time = pd.DataFrame()
df_time["时间"] = pd.date_range("2021-07-08", "2022-01-03")
date_lst = list()
date_lst.extend(pd.date_range("2021-08-01", "2021-08-18"))
date_lst.extend(pd.date_range("2021-08-25", "2021-09-01"))
date_lst.extend(pd.date_range("2021-09-10", "2021-09-10"))
date_lst.extend(pd.date_range("2021-09-29", "2021-10-10"))
date_lst.extend(pd.date_range("2021-10-27", "2021-11-11"))
date_lst.extend(pd.date_range("2021-12-01", "2021-12-12"))
date_lst.extend(pd.date_range("2021-12-31", "2022-01-01"))
lst_date = list()
for str_date in date_lst:
    lst_date.append(str(str_date).split(" ")[0])
df_time["时间"] = df_time["时间"].apply(lambda x: str(x).split(" ")[0])
df_time["是否为购物节"] = df_time["时间"].apply(lambda x: 1 if x in lst_date else 0)
backward_dates = list()
forward_dates = list()
for dt_j in range(1, 8):
    df_time["购物节_backward_" + str(dt_j)] = df_time["是否为购物节"].shift(periods=dt_j * (-1), fill_value=0)
    df_time["购物节_forward_" + str(dt_j)] = df_time["是否为购物节"].shift(periods=dt_j, fill_value=0)
    backward_dates.append("购物节_backward_" + str(dt_j))
    forward_dates.append("购物节_forward_" + str(dt_j))
tmp_backward_time_df = df_time[backward_dates]
tmp_forward_time_df = df_time[forward_dates]
df_time["未来一周购物节天数"] = tmp_backward_time_df.sum(axis=1)
df_time["前一周购物节天数"] = tmp_forward_time_df.sum(axis=1)
df_time = df_time[df_time["时间"] <= "2021-12-27"]
df_time = df_time[df_time["时间"] >= "2021-07-08"]

In [13]:
df_time.tail(20)

,时间,是否为购物节,购物节_backward_1,购物节_forward_1,购物节_backward_2,购物节_forward_2,购物节_backward_3,购物节_forward_3,购物节_backward_4,购物节_forward_4,购物节_backward_5,购物节_forward_5,购物节_backward_6,购物节_forward_6,购物节_backward_7,购物节_forward_7,未来一周购物节天数,前一周购物节天数
153,2021-12-08,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,4,7
154,2021-12-09,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,3,7
155,2021-12-10,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,2,7
156,2021-12-11,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,1,7
157,2021-12-12,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,7
158,2021-12-13,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,7
159,2021-12-14,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,6
160,2021-12-15,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,5
161,2021-12-16,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,4
162,2021-12-17,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,3


In [21]:
path_list = os.listdir(os.path.join(PATH, "dataset"))
csv_list = list()
for file_name in path_list:
    df = pd.read_csv(os.path.join(PATH, "dataset", file_name))
    df["是否为购物节"] = df_time["是否为购物节"].copy(deep=True)
    df["前一周购物节天数"] = df_time["前一周购物节天数"].copy(deep=True)
    df["未来一周购物节天数"] = df_time["未来一周购物节天数"].copy(deep=True)
    generate_dataset(df)
    csv_list.append(df)
dataset = pd.concat(csv_list, axis=0)
dataset_goods = list(dataset.商品id.unique())

In [26]:
dataset[["商品id", "时间", "总销量", "后七天平均销量"]].tail(12)

,商品id,时间,总销量,后七天平均销量
161,1449,2021-12-16,1700,1291.000000
162,1449,2021-12-17,1101,1239.571429
163,1449,2021-12-18,865,1268.285714
164,1449,2021-12-19,1582,1402.285714
165,1449,2021-12-20,1168,1664.285714
166,1449,2021-12-21,1103,1757.833333
167,1449,2021-12-22,1706,1768.200000
168,1449,2021-12-23,1512,1832.250000
169,1449,2021-12-24,741,2196.000000
170,1449,2021-12-25,1066,2761.000000


In [23]:
dic_goods = dict()
for i, goods in enumerate(dataset.商品id.unique()):
    dic_goods.setdefault(goods, i + 1)
dataset["商品id"] = dataset["商品id"].map(dic_goods)

In [31]:
df_group_by_day_sum = dataset.groupby(by=["时间"], as_index=False).sum()

In [33]:
df_day_sum = df_group_by_day_sum[["时间", "总销量", "浏览量", "直播销量", "视频销量"]].copy(deep=True)
df_day_sum.rename(columns={"总销量": "总销量_all", "浏览量": "浏览量_all", "直播销量": "直播销量_all", "视频销量": "视频销量_all"}, inplace=True)

In [34]:
dataset = pd.merge(left=dataset, right=df_day_sum, how="inner")

In [36]:
dataset["总销量总比"] = dataset.apply(lambda x: 0.0 if x["总销量_all"] == 0 else x["总销量"] / x["总销量_all"], axis=1)
dataset["浏览量总比"] = dataset.apply(lambda x: 0.0 if x["浏览量_all"] == 0 else x["浏览量"] / x["浏览量_all"], axis=1)
dataset["直播销量总比_all"] = dataset.apply(lambda x: 0.0 if x["直播销量_all"] == 0 else x["直播销量"] / x["直播销量_all"], axis=1)
dataset["视频销量总比"] = dataset.apply(lambda x: 0.0 if x["视频销量_all"] == 0 else x["视频销量"] / x["视频销量_all"], axis=1)

In [37]:
start_time = "2021-12-21"  # datetime.datetime.strptime("2021-12-21", "%Y-%m-%d").date()
end_time = "2021-12-27"  # datetime.datetime.strptime("2021-12-27", "%Y-%m-%d").date()
dataset_train = dataset[dataset["时间"] < "2021-12-25"]
dataset_test = dataset[dataset["时间"] == end_time]

In [38]:
list_sum_columns = list()
for col in dataset_train.columns:
    if col.endswith("_前第一周天平均值") or col in ['总销量', '浏览量', '抖音转化率', '视频个数', '直播个数', '直播销量', '视频销量', '视频达人', '直播达人']:
        list_sum_columns.append(col)
df_tmp_ = dataset_train[list_sum_columns]
dataset_train["是否为0"] = df_tmp_.sum(axis=1)

D:\LenovoSoftstore\Install\anaconda3\envs\od\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
dataset_train = dataset_train[dataset_train["是否为0"] >= 1e-6]

In [40]:
test_dataset = dataset_test.drop(columns=["后七天平均销量", "时间"])
dataset_train.drop(columns=["时间", "是否为0"], inplace=True)
train_dataset_x = dataset_train.drop(columns=["后七天平均销量"])
train_dataset_y = dataset_train["后七天平均销量"].copy(deep=True)

In [41]:
train_columns = train_dataset_x.columns
train_goods = list(train_dataset_x.商品id.unique())
test_goods = list(test_dataset.商品id.unique())

In [42]:
# 构建模型
params_catboost = {"iterations": 1000, "learning_rate": 0.03, "l2_leaf_reg": 3, "bagging_temperature": 1,
                   "subsample": 0.66, "random_strength": 1, "depth": 6,
                   "rsm": 1, "one_hot_max_size": 2, "leaf_estimation_method": "Gradient", "fold_len_multiplier": 2,
                   "border_count": 128, "random_seed": 2022, "loss_function": "RMSE", "eval_metric": "RMSE",
                   "od_type": "Iter", "od_wait": 50}

params_lightgbm = {"n_estimators": 300,
                   "learning_rate": 0.1,
                   "num_leaves": 500,
                   "max_depth": 6,
                   "min_data_in_leaf": 1000,
                   "lambda_l1": 1,
                   "lambda_l2": 1,
                   "min_gain_to_split": 1,
                   "bagging_fraction": 0.8,
                   "feature_fraction": 0.8,
                   "random_state": 2022,
                   "metric": "rmse"}

params_xgboost = {'booster': 'gbtree',
                  'objective': 'reg:squarederror',
                  'eval_metric': 'rmse',
                  'seed': 2022,
                  'learning_rate': 0.01,
                  'gamma': 0.1,
                  'min_child_weight': 1.1,
                  'max_depth': 5,
                  'lambda': 10,
                  'subsample': 0.7,
                  'colsample_bytree': 0.7,
                  'colsample_bylevel': 0.7,
                  'tree_method': 'exact'
                  }

In [43]:
# 构建模型
params_catboost = {"iterations": 1000, "learning_rate": 0.03, "l2_leaf_reg": 3, "bagging_temperature": 1,
                   "subsample": 0.66, "random_strength": 1, "depth": 6,
                   "rsm": 1, "one_hot_max_size": 2, "leaf_estimation_method": "Gradient", "fold_len_multiplier": 2,
                   "border_count": 128, "random_seed": 2022, "loss_function": "RMSE", "eval_metric": "RMSE",
                   "od_type": "Iter", "od_wait": 50}

params_lightgbm = {"n_estimators": 300,
                   "learning_rate": 0.1,
                   "num_leaves": 500,
                   "max_depth": 6,
                   "min_data_in_leaf": 1000,
                   "lambda_l1": 1,
                   "lambda_l2": 1,
                   "min_gain_to_split": 1,
                   "bagging_fraction": 0.8,
                   "feature_fraction": 0.8,
                   "random_state": 2022,
                   "metric": "rmse"}

params_xgboost = {'booster': 'gbtree',
                  'objective': 'reg:squarederror',
                  'eval_metric': 'rmse',
                  'seed': 2022,
                  'learning_rate': 0.01,
                  'gamma': 0.1,
                  'min_child_weight': 1.1,
                  'max_depth': 5,
                  'lambda': 10,
                  'subsample': 0.7,
                  'colsample_bytree': 0.7,
                  'colsample_bylevel': 0.7,
                  'tree_method': 'exact'
                  }

In [44]:
def model_fun(train_sample, train_label, params, model, model_name="catboost"):
    print("------------------- 训练 {} 模型 -------------------".format(model_name))
    x_train, x_test, y_train, y_test = train_test_split(train_sample, train_label, test_size=0.2, random_state=2022)
    _model = model(**params)

    print("x_train.shape : {}".format(x_train.shape))
    print("y_train.shape : {}".format(y_train.shape))
    print("x_test.shape : {}".format(x_test.shape))
    print("y_test.shape : {}".format(y_test.shape))
    _model.fit(x_train, y_train)
    if model_name.lower() == "catboost":
        _model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)], cat_features=["商品id"], verbose=50)
    elif model_name.lower() == "lightgbm":
        _model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)], eval_metric='rmse', verbose=50)
    else:
        _model.fit(x_train, y_train)

    train_pred = _model.predict(x_train)
    train_mse = mean_squared_error(y_train, train_pred)

    test_pred = _model.predict(x_test)
    test_mse = mean_squared_error(y_test, test_pred)

    result_dic = {"train_mse": train_mse, "test_mse": test_mse}
    return _model, result_dic

In [45]:
def feature_importance_model(model, model_name="catboost", cols=None):
    print("------------------------- feature_importance_model -------------------------")
    if model_name.lower() == "lightgbm":
        booster = model.booster_
        importance = booster.feature_importance(importance_type='split')
        feature_name = booster.feature_name()
    elif model_name.lower() == "catboost":
        importance = model.get_feature_importance()
        feature_name = model.feature_names_
    elif model_name == "xgboost":
        importance = model.feature_importances_
        feature_name = cols
    else:
        print("请选择正确的训练模型：[catboost, lightgbm, xgboost]")
        return pd.DataFrame()
    result = pd.DataFrame(importance, index=feature_name, columns=["FeatureImportance"])
    return result.sort_values("FeatureImportance", ascending=False)

In [46]:
# Xgboost
model_tree_xgb, model_result_xgb = model_fun(train_dataset_x, train_dataset_y, params_xgboost, xgb.XGBRegressor,
                                             "xgboost")
feature_importance_xgb = feature_importance_model(model_tree_xgb, "xgboost", train_columns)
feature_importance_xgb.head(20)

------------------- 训练 xgboost 模型 -------------------
x_train.shape : (84495, 80)
y_train.shape : (84495,)
x_test.shape : (21124, 80)
y_test.shape : (21124,)
------------------------- feature_importance_model -------------------------


,FeatureImportance
总销量_前第一周天平均值,0.105982
总销量_前第一周天方差值,0.103502
总销量_前第二周天平均值,0.100102
总销量_前第二周天方差值,0.078535
总销量_前两周最小值,0.062634
直播销量_前两周最小值,0.060711
直播销量_前第一周天方差值,0.042956
总销量,0.041661
视频销量_前第一周天方差值,0.032761
直播个数_前两周最小值,0.032591


In [52]:
def transfer_result(result_score_model, model_name: str, data_goods, _train_goods, _dic_goods):
    print("------------------------- transfer_result -------------------------")
    # res_arr = result_score_model.reshape([-1, 7])
    # lst = list()
    # for val in res_arr:
    #     lst.append(val[-1])
    lst = list(result_score_model)
    dic_col = dict(zip(data_goods, lst))
    dic_res = dict()
    for col_name in data_goods:
        if col_name in _train_goods:
            dic_res.setdefault(col_name, dic_col.get(col_name))
        else:
            dic_res.setdefault(col_name, 0.0)

    submission = pd.read_csv(os.path.join(PATH, "提交示例.csv"))
    submission["编码id"] = submission["商品id"].map(_dic_goods)
    submission["未来一周天均销量"] = submission["编码id"].map(dic_res)
    submission.drop(columns=["编码id"], inplace=True)
    submission.to_csv(os.path.join(PATH, "result_" + model_name + ".csv"), encoding="utf_8_sig", index=False)
    return submission

In [53]:

# def get_params_dic(model_name: str):
#     print("------------------------- get_params_dic -------------------------")
#
#     params_catboost = {"iterations": 1000, "learning_rate": 0.03, "l2_leaf_reg": 3, "bagging_temperature": 1,
#                        "subsample": 0.66, "random_strength": 1, "depth": 6,
#                        "rsm": 1, "one_hot_max_size": 2, "leaf_estimation_method": "Gradient", "fold_len_multiplier": 2,
#                        "border_count": 128, "random_seed": 2022, "loss_function": "RMSE", "eval_metric": "RMSE",
#                        "od_type": "Iter", "od_wait": 50}
#
#     params_lightgbm = {"n_estimators": 300,
#                        "learning_rate": 0.1,
#                        "num_leaves": 500,
#                        "max_depth": 6,
#                        "min_data_in_leaf": 1000,
#                        "lambda_l1": 1,
#                        "lambda_l2": 1,
#                        "min_gain_to_split": 1,
#                        "bagging_fraction": 0.8,
#                        "feature_fraction": 0.8,
#                        "random_state": 2022,
#                        "metric": "rmse"}
#
#     params_xgboost = {'booster': 'gbtree',
#                       'objective': 'reg:squarederror',
#                       'eval_metric': 'rmse',
#                       'seed': 2022,
#                       'learning_rate': 0.01,
#                       'gamma': 0.1,
#                       'min_child_weight': 1.1,
#                       'max_depth': 5,
#                       'lambda': 10,
#                       'subsample': 0.7,
#                       'colsample_bytree': 0.7,
#                       'colsample_bylevel': 0.7,
#                       'tree_method': 'exact'}
#
#     if model_name.lower() == "catboost":
#         if os.path.exists(os.path.join(path, "catboost_params.pkl")):
#             with open(os.path.join(path, "catboost_params.pkl"), "rb") as f:
#                 cat_params = pkl.load(f)
#             return cat_params
#         with open(os.path.join(path, "catboost_params.pkl"), "wb") as f:
#             pkl.dump(params_catboost, f)
#         return params_catboost
#     elif model_name.lower() == "lightgbm":
#         if os.path.exists(os.path.join(path, "lightgbm_params.pkl")):
#             with open(os.path.join(path, "lightgbm_params.pkl"), "rb") as f:
#                 lgbm_params = pkl.load(f)
#             return lgbm_params
#         with open(os.path.join(path, "params_lightgbm.pkl"), "wb") as f:
#             pkl.dump(params_catboost, f)
#         return params_lightgbm
#     elif model_name.lower() == "xgboost":
#         if os.path.exists(os.path.join(path, "params_xgboost.pkl")):
#             with open(os.path.join(path, "params_xgboost.pkl"), "rb") as f:
#                 params_xgboost = pkl.load(f)
#             return params_xgboost
#         with open(os.path.join(path, "params_xgboost.pkl"), "wb") as f:
#             pkl.dump(params_xgboost, f)
#         return params_xgboost
#     else:
#         print("-----------> 数据的模型名称不在 {catboost, lightgbm, xgboost}, 请重新输入")
#         return dict()


# catboost模型调参
def model_catboost(dic_param):
    print("------------------------- model_catboost -------------------------")

    def catboost_cv(iterations, learning_rate, depth, subsample, l2_leaf_reg):
        cat_params = {"iterations": 1000, "learning_rate": 0.03, "l2_leaf_reg": 3, "bagging_temperature": 1,
                      "subsample": 0.66, "random_strength": 1, "depth": 6,
                      "rsm": 1, "one_hot_max_size": 2, "leaf_estimation_method": "Gradient", "fold_len_multiplier": 2,
                      "border_count": 128, "random_seed": 2022, "loss_function": "RMSE", "eval_metric": "RMSE",
                      "od_type": "Iter", "od_wait": 50}
        cat_params.update(
            {"iterations": int(iterations), "depth": int(depth), "learning_rate": learning_rate, "subsample": subsample,
             "l2_leaf_reg": int(l2_leaf_reg)})
        _, result = model_fun(dic_param["train_sample"], dic_param["train_label"], cat_params, dic_param["model"],
                              dic_param["model_name"])
        return result.get("test_mse")

    cat_bayes = BayesianOptimization(catboost_cv, dic_param["param_value_dics"], random_state=2022)
    cat_bayes.maximize(init_points=1, n_iter=25)
    return cat_bayes.max.get("params")


# lightgbm模型调参
def model_lightgbm(dic_param):
    print("------------------------- model_lightgbm -------------------------")

    def lightgbm_cv(n_estimators, learning_rate, max_depth, feature_fraction, bagging_fraction):
        lgbm_params = {"n_estimators": 300,
                       "learning_rate": 0.1,
                       "num_leaves": 500,
                       "max_depth": 6,
                       "min_data_in_leaf": 1000,
                       "lambda_l1": 1,
                       "lambda_l2": 1,
                       "min_gain_to_split": 1,
                       "bagging_fraction": 0.8,
                       "feature_fraction": 0.8,
                       "random_state": 2022,
                       "metric": "rmse"}
        lgbm_params.update(
            {"n_estimators": int(n_estimators), "max_depth": int(max_depth), "learning_rate": learning_rate,
             "feature_fraction": feature_fraction,
             "bagging_fraction": bagging_fraction})
        _, result = model_fun(dic_param["train_sample"], dic_param["train_label"], lgbm_params, dic_param["model"],
                              dic_param["model_name"])
        return result.get("test_mse")

    lightgbm_bayes = BayesianOptimization(lightgbm_cv, dic_param["param_value_dics"], random_state=2022)
    lightgbm_bayes.maximize(init_points=1, n_iter=25)
    return lightgbm_bayes.max.get("params")


# xgboost模型调参
def model_xgboost(dic_param):
    print("------------------------- model_xgboost -------------------------")

    def xgboost_cv(n_estimators, learning_rate, max_depth, colsample_bytree, colsample_bylevel, gamma):
        xgb_params = {'booster': 'gbtree',
                      'objective': 'reg:squarederror',
                      'eval_metric': 'rmse',
                      'seed': 2022,
                      'learning_rate': 0.01,
                      'gamma': 0.1,
                      'min_child_weight': 1.1,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'tree_method': 'exact'
                      }
        xgb_params.update(
            {"n_estimators": int(n_estimators), "max_depth": int(max_depth), "learning_rate": learning_rate,
             "colsample_bytree": colsample_bytree,
             "colsample_bylevel": colsample_bylevel,
             "gamma": gamma})
        _, result = model_fun(dic_param["train_sample"], dic_param["train_label"], xgb_params, dic_param["model"],
                              dic_param["model_name"])
        return result.get("test_mse")

    xgboost_bayes = BayesianOptimization(xgboost_cv, dic_param["param_value_dics"], random_state=2022)
    xgboost_bayes.maximize(init_points=1, n_iter=25)
    return xgboost_bayes.max.get("params")


# 模型预测
def get_result(params_dic, model, model_name: str, x_data, y_data, test_data):
    print("------------------------- get_result -------------------------")
    ans = []
    mean_score = 0
    sk = KFold(n_splits=10, shuffle=True, random_state=2022)
    _model = model(**params_dic)
    for train_index, test_index in sk.split(x_data, y_data):
        x_train = x_data.iloc[train_index]
        y_train = y_data.iloc[train_index]
        x_test = x_data.iloc[test_index]
        y_test = y_data.iloc[test_index]
        if model_name.lower() == "catboost":
            regressor_model = _model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=500,
                                         cat_features=["商品id"])
        elif model_name.lower() == "lightgbm":
            regressor_model = _model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=500)
        else:
            regressor_model = _model.fit(x_train, y_train)
        y_pred = regressor_model.predict(x_test)
        test_mse = mean_squared_error(y_test, y_pred)
        print("model 验证MSE：{}".format(test_mse))
        mean_score += test_mse / 10.
        y_test_pred = regressor_model.predict(test_data)
        ans.append(y_test_pred)
    print("10折平均MSE：{}".format(mean_score))
    model_pred = sum(ans) / 10.
    return model_pred

In [54]:
# catboost
catboost_params_dic = {"iterations": (50, 600), "depth": (3, 10), "learning_rate": (0.02, 0.3),
                       "subsample": (0.6, 1.0),
                       "l2_leaf_reg": (1, 10)}
catboost_bayes_dic = {"param_value_dics": catboost_params_dic, "train_sample": train_dataset_x,
                      "train_label": train_dataset_y, "model": CatBoostRegressor, "model_name": "catboost"}
catboost_best_params = model_catboost(catboost_bayes_dic)

------------------------- model_catboost -------------------------
|   iter    |  target   |   depth   | iterat... | l2_lea... | learni... | subsample |
-------------------------------------------------------------------------------------
------------------- 训练 catboost 模型 -------------------
x_train.shape : (84495, 80)
y_train.shape : (84495,)
x_test.shape : (21124, 80)
y_test.shape : (21124,)
0:	learn: 4808.5461907	total: 13.1ms	remaining: 4.25s
1:	learn: 4728.8647214	total: 19.2ms	remaining: 3.08s
2:	learn: 4649.2536341	total: 25.3ms	remaining: 2.71s
3:	learn: 4578.8758657	total: 32.1ms	remaining: 2.57s
4:	learn: 4507.2355308	total: 38.8ms	remaining: 2.47s
5:	learn: 4435.6003636	total: 44.7ms	remaining: 2.37s
6:	learn: 4369.4960157	total: 51.1ms	remaining: 2.31s
7:	learn: 4304.9433153	total: 58.1ms	remaining: 2.29s
8:	learn: 4245.3337884	total: 64.8ms	remaining: 2.27s
9:	learn: 4182.0474844	total: 70.5ms	remaining: 2.21s
10:	learn: 4131.4648042	total: 76.9ms	remaining: 2.19s
11:	lea

In [55]:
catboost_best_params["depth"] = round(catboost_best_params["depth"])
catboost_best_params["iterations"] = round(catboost_best_params["iterations"])
catboost_best_params["learning_rate"] = round(catboost_best_params["learning_rate"], 4)
catboost_best_params["l2_leaf_reg"] = round(catboost_best_params["l2_leaf_reg"])
catboost_best_params["subsample"] = round(catboost_best_params["subsample"], 4)
params_catboost.update(catboost_best_params)

with open(os.path.join(PATH, "catboost_best_params.pkl"), "wb") as f:
    pkl.dump(params_catboost, f)
result_score_cat = get_result(params_catboost, CatBoostRegressor, "catboost", train_dataset_x, train_dataset_y,
                              test_dataset)
result_cat = transfer_result(result_score_cat, "catboost", test_goods, train_goods, dic_goods)
model_cat, result_dic_cat = model_fun(train_dataset_x, train_dataset_y, params_catboost, CatBoostRegressor, "catboost")
feature_importance_cat = feature_importance_model(model_cat, "catboost", train_columns)
feature_importance_cat.to_csv(os.path.join(PATH, "feature_importance_cat.csv"), encoding="utf_8_sig")

------------------------- get_result -------------------------
0:	learn: 5045.7292061	test: 5090.1524895	best: 5090.1524895 (0)	total: 33.9ms	remaining: 2.1s
62:	learn: 2563.8568954	test: 2490.5543046	best: 2490.5543046 (62)	total: 1.75s	remaining: 0us

bestTest = 2490.554305
bestIteration = 62

model 验证MSE：6202860.530715625
0:	learn: 4824.6648686	test: 6709.7790080	best: 6709.7790080 (0)	total: 48.4ms	remaining: 3s
62:	learn: 2532.6583829	test: 3369.3166104	best: 3369.3166104 (62)	total: 1.73s	remaining: 0us

bestTest = 3369.31661
bestIteration = 62

model 验证MSE：11352294.394434473
0:	learn: 5068.2218642	test: 4867.8911375	best: 4867.8911375 (0)	total: 85.3ms	remaining: 5.29s
62:	learn: 2547.0642248	test: 2852.8037506	best: 2852.8037506 (62)	total: 1.76s	remaining: 0us

bestTest = 2852.803751
bestIteration = 62

model 验证MSE：8138489.02570332
0:	learn: 5159.7932697	test: 3867.8479420	best: 3867.8479420 (0)	total: 91ms	remaining: 5.64s
62:	learn: 2557.9801531	test: 2324.4705806	best: 2324

In [56]:
# lightgbm
lightgbm_params_dic = {"n_estimators": (50, 2000), "max_depth": (3, 10), "learning_rate": (0.02, 0.3),
                       "feature_fraction": (0.6, 1.0),
                       "bagging_fraction": (0.6, 1.)}
lightgbm_bayes_dic = {"param_value_dics": lightgbm_params_dic, "train_sample": train_dataset_x,
                      "train_label": train_dataset_y, "model": lgb.LGBMRegressor, "model_name": "lightgbm"}
lightgbm_best_params = model_lightgbm(lightgbm_bayes_dic)

lightgbm_best_params["max_depth"] = round(lightgbm_best_params["max_depth"])
lightgbm_best_params["n_estimators"] = round(lightgbm_best_params["n_estimators"])
lightgbm_best_params["learning_rate"] = round(lightgbm_best_params["learning_rate"], 4)
lightgbm_best_params["feature_fraction"] = round(lightgbm_best_params["feature_fraction"], 4)
lightgbm_best_params["bagging_fraction"] = round(lightgbm_best_params["bagging_fraction"], 4)
params_lightgbm.update(lightgbm_best_params)
with open(os.path.join(PATH, "lightgbm_best_params.pkl"), "wb") as f:
    pkl.dump(params_lightgbm, f)
result_score_lgbm = get_result(params_lightgbm, lgb.LGBMRegressor, "lightgbm", train_dataset_x, train_dataset_y,
                               test_dataset)
result_lgbm = transfer_result(result_score_lgbm, "lightgbm", test_goods, train_goods, dic_goods)

model_lgbm, result_dic_lgbm = model_fun(train_dataset_x, train_dataset_y, params_lightgbm, lgb.LGBMRegressor,
                                        "lightgbm")
feature_importance_lgbm = feature_importance_model(model_lgbm, "lightgbm", train_columns)
feature_importance_lgbm.to_csv(os.path.join(PATH, "feature_importance_lgbm.csv"), encoding="utf_8_sig")

------------------------- model_lightgbm -------------------------
|   iter    |  target   | baggin... | featur... | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------------------
------------------- 训练 lightgbm 模型 -------------------
x_train.shape : (84495, 80)
y_train.shape : (84495,)
x_test.shape : (21124, 80)
y_test.shape : (21124,)
[LightGBM] [Warning] feature_fraction is set=0.7996231243407952, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7996231243407952
[LightGBM] [Warning] min_data_in_leaf is set=1000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1000
[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] bagging_fraction is set=0.6037434455231059, subsample=1.0 will be ignored. Current

D:\LenovoSoftstore\Install\anaconda3\envs\od\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's rmse: 3569.17	valid_1's rmse: 4795.13
[100]	training's rmse: 3405.95	valid_1's rmse: 4571.44
[150]	training's rmse: 3305.31	valid_1's rmse: 4425.56
[200]	training's rmse: 3228.96	valid_1's rmse: 4312.71
[250]	training's rmse: 3172.16	valid_1's rmse: 4227.71
[300]	training's rmse: 3127.92	valid_1's rmse: 4156.83
[350]	training's rmse: 3090.56	valid_1's rmse: 4094.83
[400]	training's rmse: 3053.9	valid_1's rmse: 4033.31
[450]	training's rmse: 3025.94	valid_1's rmse: 3986.24
[500]	training's rmse: 2999.49	valid_1's rmse: 3941.35
[550]	training's rmse: 2973.76	valid_1's rmse: 3898.34
[600]	training's rmse: 2953.74	valid_1's rmse: 3861.09
[650]	training's rmse: 2931.05	valid_1's rmse: 3822.47
[700]	training's rmse: 2911.16	valid_1's rmse: 3786.99
[750]	training's rmse: 2892.4	valid_1's rmse: 3754.03
[800]	training's rmse: 2875.3	valid_1's rmse: 3724.1
[850]	training's rmse: 2859.42	valid_1's rmse: 3695.24
[900]	training's rmse: 2843.69	valid_1's rmse: 3667.35
[950]	training'

D:\LenovoSoftstore\Install\anaconda3\envs\od\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's rmse: 3134.07	valid_1's rmse: 4172.3
[100]	training's rmse: 2948.37	valid_1's rmse: 3872.18
[150]	training's rmse: 2836.55	valid_1's rmse: 3680.5
[200]	training's rmse: 2744.01	valid_1's rmse: 3523.8
[250]	training's rmse: 2660.97	valid_1's rmse: 3395.08
[300]	training's rmse: 2593.42	valid_1's rmse: 3284.32
[350]	training's rmse: 2525.24	valid_1's rmse: 3182.2
[400]	training's rmse: 2469.05	valid_1's rmse: 3095.45
[450]	training's rmse: 2420.08	valid_1's rmse: 3014.69
[500]	training's rmse: 2376.13	valid_1's rmse: 2949.55
[550]	training's rmse: 2337.05	valid_1's rmse: 2889
[600]	training's rmse: 2294.63	valid_1's rmse: 2829.12
[650]	training's rmse: 2259.14	valid_1's rmse: 2771.24
[700]	training's rmse: 2224.06	valid_1's rmse: 2717.42
[750]	training's rmse: 2192.1	valid_1's rmse: 2669.17
[800]	training's rmse: 2160	valid_1's rmse: 2622.6
[850]	training's rmse: 2133.73	valid_1's rmse: 2581.02
[900]	training's rmse: 2106.35	valid_1's rmse: 2538.56
[950]	training's rmse: 

...

model 验证MSE：8560407.355402825
[LightGBM] [Warning] feature_fraction is set=0.755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.755
[LightGBM] [Warning] min_data_in_leaf is set=1000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1000
[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] bagging_fraction is set=0.6631, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6631
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1


D:\LenovoSoftstore\Install\anaconda3\envs\od\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


model 验证MSE：19691526.844549764
10折平均MSE：13315259.02018425
------------------------- transfer_result -------------------------
------------------- 训练 lightgbm 模型 -------------------
x_train.shape : (84495, 80)
y_train.shape : (84495,)
x_test.shape : (21124, 80)
y_test.shape : (21124,)
[LightGBM] [Warning] feature_fraction is set=0.755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.755
[LightGBM] [Warning] min_data_in_leaf is set=1000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1000
[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] bagging_fraction is set=0.6631, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6631
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1


D:\LenovoSoftstore\Install\anaconda3\envs\od\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's rmse: 3701.77	valid_1's rmse: 4965.74
[100]	training's rmse: 3511.32	valid_1's rmse: 4735.94
------------------------- feature_importance_model -------------------------


In [57]:
 # xgboost
xgboost_params_dic = {"n_estimators": (50, 2000), "max_depth": (3, 10), "learning_rate": (0.02, 0.3),
                      "colsample_bytree": (0.5, 1.0),
                      "colsample_bylevel": (0.6, 1.),
                      "gamma": (0.01, 1.0)}
xgboost_bayes_dic = {"param_value_dics": xgboost_params_dic, "train_sample": train_dataset_x,
                     "train_label": train_dataset_y, "model": xgb.XGBRegressor, "model_name": "xgboost"}
xgboost_best_params = model_xgboost(xgboost_bayes_dic)

xgboost_best_params["max_depth"] = round(xgboost_best_params["max_depth"])
xgboost_best_params["n_estimators"] = round(xgboost_best_params["n_estimators"])
xgboost_best_params["learning_rate"] = round(xgboost_best_params["learning_rate"], 4)
xgboost_best_params["colsample_bytree"] = round(xgboost_best_params["colsample_bytree"], 4)
xgboost_best_params["colsample_bylevel"] = round(xgboost_best_params["colsample_bylevel"], 4)
xgboost_best_params["gamma"] = round(xgboost_best_params["gamma"], 4)

params_xgboost.update(xgboost_best_params)
with open(os.path.join(PATH, "xgboost_best_params.pkl"), "wb") as f:
    pkl.dump(params_xgboost, f)

result_score_xgb = get_result(params_xgboost, xgb.XGBRegressor, "xgboost", train_dataset_x, train_dataset_y,
                              test_dataset)
result_xgb = transfer_result(result_score_xgb, "xgboost", test_goods, train_goods, dic_goods)

model_xgb, result_dic_xgb = model_fun(train_dataset_x, train_dataset_y, params_xgboost, xgb.XGBRegressor,
                                      "xgboost")
feature_importance_xgb = feature_importance_model(model_xgb, "xgboost", train_columns)
feature_importance_xgb.to_csv(os.path.join(PATH, "feature_importance_xgb.csv"), encoding="utf_8_sig")

------------------------- model_xgboost -------------------------
|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------------------------------
------------------- 训练 xgboost 模型 -------------------
x_train.shape : (84495, 80)
y_train.shape : (84495,)
x_test.shape : (21124, 80)
y_test.shape : (21124,)
|  1        |  2.631e+0 |  0.6037   |  0.7495   |  0.1222   |  0.03399  |  7.798    |  999.6    |
------------------- 训练 xgboost 模型 -------------------
x_train.shape : (84495, 80)
y_train.shape : (84495,)
x_test.shape : (21124, 80)
y_test.shape : (21124,)
|  2        |  2.642e+0 |  0.9914   |  0.7455   |  0.102    |  0.07926  |  9.066    |  238.1    |
------------------- 训练 xgboost 模型 -------------------
x_train.shape : (84495, 80)
y_train.shape : (84495,)
x_test.shape : (21124, 80)
y_test.shape : (21124,)
|  3        |  3.84e+06 |  0.6027   |  0.794    |  0.5514   |  0.027

In [67]:
result_fuse = pd.read_csv(os.path.join(PATH, "提交示例.csv"))
result_fuse["未来一周天均销量"] = (result_cat["未来一周天均销量"] + result_lgbm["未来一周天均销量"] + result_xgb["未来一周天均销量"]) / 3.8
result_fuse.to_csv(os.path.join(PATH, "result_fuse.csv"), encoding="utf_8_sig", index=False)

In [68]:
result_1764 = pd.read_csv(os.path.join(PATH, "1764.csv"))
np.sqrt(mean_squared_error(result_fuse["未来一周天均销量"], result_1764["未来一周天均销量"]))

518.9771338752354